In [1]:
library(swat)
library(ggplot2)
library(reshape2)
library(xgboost)
library(caret)
library(repr)
library(dplyr)
options(repr.plot.width=5.5, repr.plot.height=5.5)

SWAT 1.4.1
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Loading required package: lattice

Attaching package: ‘dplyr’

The following object is masked from ‘package:xgboost’:

    slice

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
Sys.setenv(CAS_CLIENT_SSL_CA_LIST = "/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem")
conn <- CAS('frasepviya35smp', 5570)
# Activate metric tracing and other session parameters
cas.sessionProp.setSessOpt(conn, metrics=TRUE, timeout=1800, caslib='casuser')

NOTE: Connecting to CAS and generating CAS action functions for loaded
      action sets...
NOTE: To generate the functions with signatures (for tab completion), set 
      options(cas.gen.function.sig=TRUE).
NOTE: 'CASUSER(viyademo01)' is now the active caslib.
NOTE: Action 'sessionProp.setSessOpt' used (Total process time):
NOTE:       real time               0.003679 seconds
NOTE:       cpu time                0.003602 seconds (97.91%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            122.80G
NOTE:       memory                  400.81K (0.00%)


list()

In [3]:
# Read in the hmeq CSV to an in-memory data table and create a CAS table object reference
castbl <- cas.read.csv(conn, 'http://support.sas.com/documentation/onlinedoc/viya/exampledatasets/hmeq.csv')

NOTE: Executing action 'table.columnInfo'.
NOTE: Action 'table.columnInfo' used (Total process time):
NOTE:       real time               0.001105 seconds
NOTE:       cpu time                0.001030 seconds (93.21%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            122.80G
NOTE:       memory                  831.63K (0.00%)


In [13]:
    loadActionSet(conn, "dataSciencePilot")
    target <<- "BAD"
    tbl_name <<- "HMEQ"

    features <- cas.dataSciencePilot.featureMachine(
        conn,
        table                 = list(name =tbl_name),
        target                = "BAD",
        transformationPolicy  = list(missing = TRUE),
        transformationOut     = list(name= "TRANSFORMATION_OUT", replace = TRUE),
        featureOut            = list(name= "FEATURE_OUT", replace = TRUE),
        casout                = list(name= paste0(tbl_name,"_TRANSFORMED"), replace = TRUE),
        saveState             = list(name= "ASTORE_OUT", replace = TRUE)
    )



NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'dataSciencePilot'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.000559 seconds
NOTE:       cpu time                0.000494 seconds (88.37%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            122.80G
NOTE:       memory                  209.81K (0.00%)
NOTE: Executing action 'builtins.actionSetInfo'.
NOTE: Action 'builtins.actionSetInfo' used (Total process time):
NOTE:       real time               0.000962 seconds
NOTE:       cpu time                0.000919 seconds (95.53%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            122.80G
NOTE:       memory                  317.09K (0.00%)
NOTE: Executing action 'builtins.listActions'.
NOTE: Information for action set 'dataSciencePilot':
NOTE:    dataSciencePilot
NOTE:       exploreData - Exploration, automatic variable analysis and grouping usi

In [ ]:
results <– cas.dataSciencePilot.featureMachine(s,
casout=list(
caslib="string",
indexVars=list("variable-name-1" <, "variable-name-2", ...>),
lifetime=64-bit-integer,
name="table-name",
promote=TRUE | FALSE,
replace=TRUE | FALSE
),
copyVars=list("variable-name-1" <, "variable-name-2", ...>),
distinctCountLimit=integer,
ecdfTolerance=double,
event="string",
explorationPolicy=list(
cardinality=list(
lowMediumCutoff=double
mediumHighCutoff=double
minNObsPerTargetLevel=double
),
cv=list(
lowMoment=double
lowRobust=double
),
dateTimeVariables=list("variable-name-1" <, "variable-name-2", ...>),
dateVariables=list("variable-name-1" <, "variable-name-2", ...>),
entropy=list(
giniLowMediumCutoff=double
giniMediumHighCutoff=double
shannonLowMediumCutoff=double
shannonMediumHighCutoff=double
),
iqv=list(
highTopBottom=double
highTopTwo=double
highVariationRatio=double
),
kurtosis=list(
momentLowMediumCutoff=double
momentMediumHighCutoff=double
robustLowMediumCutoff=double
robustMediumHighCutoff=double
),
missing=list(
lowMediumCutoff=double
mediumHighCutoff=double
),
nominal=list(
cardinalityRatio=double
cardinalityThreshold=double
includeNegative=TRUE | FALSE
includeNonIntegral=TRUE | FALSE
intervals=list("variable-name-1" <, "variable-name-2", ...>)
nominals=list("variable-name-1" <, "variable-name-2", ...>)
),
outlier=list(
momentLowMediumCutoff=double
momentMediumHighCutoff=double
robustLowMediumCutoff=double
robustMediumHighCutoff=double
),
skewness=list(
momentLowMediumCutoff=double
momentMediumHighCutoff=double
robustLowMediumCutoff=double
robustMediumHighCutoff=double
),
timeVariables=list("variable-name-1" <, "variable-name-2", ...>)
),
*required parameter featureOut=list(
caslib="string",
indexVars=list("variable-name-1" <, "variable-name-2", ...>),
lifetime=64-bit-integer,
name="table-name",
promote=TRUE | FALSE,
replace=TRUE | FALSE
),
freq="variable-name",
inputs=list( list(
format="string",
formattedLength=integer,
label="string",
*required parameter name="variable-name",
nfd=integer,
nfl=integer
) <, list(...)>),
misraGries=TRUE | FALSE,
rankPolicy=list(
intervalStat="AVGQUANKURT" | "AVGQUANSKEW" | "CLASSICALKURT" | "CLASSICALSKEW" | "ENTROPY" | "MI" | "NORMMI" | "PEARSON" | "SU",
nominalStat="CHISQ" | "CRAMERSV" | "ENTROPY" | "FTEST" | "G2" | "IV" | "MI" | "NORMMI" | "SU",
separateRank=TRUE | FALSE,
topKInteractions=integer,
topKSave=integer
),
saveState=list(
caslib="string",
indexVars=list("variable-name-1" <, "variable-name-2", ...>),
lifetime=64-bit-integer,
name="table-name",
promote=TRUE | FALSE,
replace=TRUE | FALSE
),
screenPolicy=list(
constant=TRUE | FALSE,
groupRareLevels=TRUE | FALSE,
leakagePercentThreshold=double,
lowCv=TRUE | FALSE,
lowMutualInformation=double,
missingIndicatorPercent=double,
missingPercentThreshold=double,
redundant=double
),
seed=integer,
*required parameter table=list(
caslib="string",
computedOnDemand=TRUE | FALSE,
computedVars=list( list(
format="string",
formattedLength=integer,
label="string",
*required parameter name="variable-name",
nfd=integer,
nfl=integer
) <, list(...)>),
computedVarsProgram="string",
dataSourceOptions=list(key-1=list(any-list-or-data-type-1) <, key-2=list(any-list-or-data-type-2), ...>),
importOptions=list(fileType="ANY" | "AUDIO" | "AUTO" | "BASESAS" | "CSV" | "DOCUMENT" | "DTA" | "ESP" | "EXCEL" | "FMT" | "HDAT" | "IMAGE" | "JMP" | "LASR" | "SPSS" | "VIDEO" | "XLS", fileType-specific-parameters),
*required parameter name="table-name",
singlePass=TRUE | FALSE,
vars=list( list(
format="string",
formattedLength=integer,
label="string",
*required parameter name="variable-name",
nfd=integer,
nfl=integer
) <, list(...)>),
where="where-expression",
whereTable=list(
casLib="string"
dataSourceOptions=list(key-1=list(any-list-or-data-type-1) <, key-2=list(any-list-or-data-type-2), ...>)
importOptions=list(fileType="ANY" | "AUDIO" | "AUTO" | "BASESAS" | "CSV" | "DOCUMENT" | "DTA" | "ESP" | "EXCEL" | "FMT" | "HDAT" | "IMAGE" | "JMP" | "LASR" | "SPSS" | "VIDEO" | "XLS", fileType-specific-parameters)
*required parameter name="table-name"
vars=list( list(
format="string",
formattedLength=integer,
label="string",
*required parameter name="variable-name",
nfd=integer,
nfl=integer
) <, list(...)>)
where="where-expression"
)
),
*required parameter target="variable-name",
*required parameter transformationOut=list(
caslib="string",
indexVars=list("variable-name-1" <, "variable-name-2", ...>),
lifetime=64-bit-integer,
name="table-name",
promote=TRUE | FALSE,
replace=TRUE | FALSE
),
transformationPolicy=list(
cardinality=TRUE | FALSE,
entropy=TRUE | FALSE,
interaction=TRUE | FALSE,
iqv=TRUE | FALSE,
kurtosis=TRUE | FALSE,
missing=TRUE | FALSE,
outlier=TRUE | FALSE,
polynomial=TRUE | FALSE,
skewness=TRUE | FALSE
)
)

In [9]:
 cas.builtins.help(conn,actionset="dataSciencePilot",action="featureMachine")

NOTE: Executing action 'builtins.help'.
NOTE: Information for action 'dataSciencePilot.featureMachine':
NOTE: The following parameters are accepted.  Default values are shown.
NOTE:    list table={
NOTE:       specifies the table name, caslib, and other common parameters.
NOTE:       string name=NULL (required),
NOTE:       specifies the name of the input table.
NOTE:       string caslib=NULL,
NOTE:       specifies the caslib for the input table that you want to use with the action. By default, the active caslib is used. Specify a value only if you need to access a table from a different caslib.
NOTE:       string where=NULL,
NOTE:       specifies an expression for subsetting the input data.
NOTE:       array of computedVars={
NOTE:       specifies the names of the computed variables to create. Specify an expression for each variable in the computedVarsProgram parameter. If you do not specify this parameter, then all variables from computedVarsProgram are automatically included.
NOTE: 

$featureMachine
[1] 1